In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np
from pylab import rcParams

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_recall_curve
from sklearn.metrics import recall_score, classification_report, auc, roc_curve
from sklearn.metrics import precision_recall_fscore_support, f1_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

from sklearn.model_selection import train_test_split

SEED = 123 #used to help randomly select the data points
DATA_SPLIT_PCT = 0.2

rcParams['figure.figsize'] = 8, 6
LABELS = ["Normal","Break"]

In [3]:
df = pd.read_csv("bosch_selected_without_null.csv") 
df

,Unnamed: 0,L1_S24_F1622,L1_S24_F1723,L1_S24_F1166,L1_S24_F1846,L0_S3_F96,L3_S33_F3859,L3_S33_F3857,L1_S24_F1844,L1_S24_F1609,...,L1_S25_F2237,L3_S30_F3574,L3_S30_F3769,L1_S25_F2925,L1_S25_F2484,L3_S34_F3880,L3_S41_F4020,L0_S14_F370,L2_S26_F3113,L1_S24_F844
0,0,0.000018,0.000292,0.001569,-0.006682,-0.000358,-0.048,-0.102,0.001034,0.005514,...,0.035576,0.001711,-0.007,0.001233,0.016188,0.0,0.008414,-0.005228,-0.001004,-0.019477
1,1,0.000018,0.000292,0.001569,-0.006682,-0.000515,-0.210,-0.026,0.001034,0.005514,...,0.035576,0.033000,-0.024,0.001233,0.016188,0.0,0.008414,-0.324000,-0.001004,-0.019477
2,2,0.000018,0.000292,0.001569,-0.006682,-0.000515,0.236,0.191,0.001034,0.005514,...,0.035576,-0.147000,-0.026,0.001233,0.016188,0.0,0.008414,-0.324000,-0.001011,-0.019477
3,3,0.000018,0.000292,0.001569,-0.006682,-0.000515,0.165,0.031,0.001034,0.005514,...,0.035576,-0.054000,0.022,0.001233,0.016188,0.0,0.008414,-0.324000,-0.001011,-0.019477
4,4,0.000018,0.000292,0.001569,-0.006682,0.311000,0.147,-0.049,0.001034,0.005514,...,0.035576,0.119000,0.022,0.001233,0.016188,0.0,0.008746,-0.324000,-0.001011,-0.019477
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1183742,1183742,-0.000142,-0.005846,-0.014684,0.004028,-0.005042,0.152,-0.095,0.002243,0.000259,...,0.045373,-0.168000,-0.043,-0.005163,-0.025226,0.0,0.019205,-0.019136,-0.108000,0.008068
1183743,1183743,-0.000142,-0.005846,-0.014684,0.004028,-0.005195,-0.139,-0.184,0.002243,0.000259,...,0.045373,0.141000,-0.004,-0.005163,-0.025226,0.0,0.019205,-0.019136,0.007303,0.008068
1183744,1183744,-0.000142,-0.005846,-0.014684,0.004028,-0.245000,-0.114,0.019,0.002243,0.000259,...,0.045373,-0.154000,-0.001,-0.005163,-0.025226,0.0,0.019205,-0.018788,0.007303,0.008068
1183745,1183745,-0.000142,-0.005846,-0.014684,0.004028,-0.256000,0.031,0.072,0.002243,0.000259,...,0.045373,0.119000,-0.001,-0.005163,-0.025226,0.0,0.019205,-0.018788,0.007303,0.008068


In [4]:
df = df.drop(['Unnamed: 0'], axis=1)

In [5]:
df_response= pd.read_csv('bosch_response.csv')
df_response = df_response.rename(columns={'0.1': 'Response'})
df_response
df_response = df_response.drop(['0'], axis=1)
df_response

,Response
0,0
1,0
2,0
3,0
4,0
...,...
1183741,0
1183742,0
1183743,0
1183744,0


In [6]:
df_new=df.join(df_response)
df_new

,L1_S24_F1622,L1_S24_F1723,L1_S24_F1166,L1_S24_F1846,L0_S3_F96,L3_S33_F3859,L3_S33_F3857,L1_S24_F1844,L1_S24_F1609,L1_S24_F1474,...,L3_S30_F3574,L3_S30_F3769,L1_S25_F2925,L1_S25_F2484,L3_S34_F3880,L3_S41_F4020,L0_S14_F370,L2_S26_F3113,L1_S24_F844,Response
0,0.000018,0.000292,0.001569,-0.006682,-0.000358,-0.048,-0.102,0.001034,0.005514,0.001974,...,0.001711,-0.007,0.001233,0.016188,0.0,0.008414,-0.005228,-0.001004,-0.019477,0.0
1,0.000018,0.000292,0.001569,-0.006682,-0.000515,-0.210,-0.026,0.001034,0.005514,0.001974,...,0.033000,-0.024,0.001233,0.016188,0.0,0.008414,-0.324000,-0.001004,-0.019477,0.0
2,0.000018,0.000292,0.001569,-0.006682,-0.000515,0.236,0.191,0.001034,0.005514,0.001974,...,-0.147000,-0.026,0.001233,0.016188,0.0,0.008414,-0.324000,-0.001011,-0.019477,0.0
3,0.000018,0.000292,0.001569,-0.006682,-0.000515,0.165,0.031,0.001034,0.005514,0.001974,...,-0.054000,0.022,0.001233,0.016188,0.0,0.008414,-0.324000,-0.001011,-0.019477,0.0
4,0.000018,0.000292,0.001569,-0.006682,0.311000,0.147,-0.049,0.001034,0.005514,0.001974,...,0.119000,0.022,0.001233,0.016188,0.0,0.008746,-0.324000,-0.001011,-0.019477,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1183742,-0.000142,-0.005846,-0.014684,0.004028,-0.005042,0.152,-0.095,0.002243,0.000259,-0.018253,...,-0.168000,-0.043,-0.005163,-0.025226,0.0,0.019205,-0.019136,-0.108000,0.008068,0.0
1183743,-0.000142,-0.005846,-0.014684,0.004028,-0.005195,-0.139,-0.184,0.002243,0.000259,-0.018253,...,0.141000,-0.004,-0.005163,-0.025226,0.0,0.019205,-0.019136,0.007303,0.008068,0.0
1183744,-0.000142,-0.005846,-0.014684,0.004028,-0.245000,-0.114,0.019,0.002243,0.000259,-0.018253,...,-0.154000,-0.001,-0.005163,-0.025226,0.0,0.019205,-0.018788,0.007303,0.008068,0.0
1183745,-0.000142,-0.005846,-0.014684,0.004028,-0.256000,0.031,0.072,0.002243,0.000259,-0.018253,...,0.119000,-0.001,-0.005163,-0.025226,0.0,0.019205,-0.018788,0.007303,0.008068,0.0


In [7]:
df_new = df_new[df_new['Response'].notna()]
df_new

,L1_S24_F1622,L1_S24_F1723,L1_S24_F1166,L1_S24_F1846,L0_S3_F96,L3_S33_F3859,L3_S33_F3857,L1_S24_F1844,L1_S24_F1609,L1_S24_F1474,...,L3_S30_F3574,L3_S30_F3769,L1_S25_F2925,L1_S25_F2484,L3_S34_F3880,L3_S41_F4020,L0_S14_F370,L2_S26_F3113,L1_S24_F844,Response
0,0.000018,0.000292,0.001569,-0.006682,-0.000358,-0.048,-0.102,0.001034,0.005514,0.001974,...,0.001711,-0.007,0.001233,0.016188,0.0,0.008414,-0.005228,-0.001004,-0.019477,0.0
1,0.000018,0.000292,0.001569,-0.006682,-0.000515,-0.210,-0.026,0.001034,0.005514,0.001974,...,0.033000,-0.024,0.001233,0.016188,0.0,0.008414,-0.324000,-0.001004,-0.019477,0.0
2,0.000018,0.000292,0.001569,-0.006682,-0.000515,0.236,0.191,0.001034,0.005514,0.001974,...,-0.147000,-0.026,0.001233,0.016188,0.0,0.008414,-0.324000,-0.001011,-0.019477,0.0
3,0.000018,0.000292,0.001569,-0.006682,-0.000515,0.165,0.031,0.001034,0.005514,0.001974,...,-0.054000,0.022,0.001233,0.016188,0.0,0.008414,-0.324000,-0.001011,-0.019477,0.0
4,0.000018,0.000292,0.001569,-0.006682,0.311000,0.147,-0.049,0.001034,0.005514,0.001974,...,0.119000,0.022,0.001233,0.016188,0.0,0.008746,-0.324000,-0.001011,-0.019477,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1183741,-0.000142,-0.038000,-0.014684,0.058000,-0.005042,0.054,0.033,0.261000,0.000259,-0.018253,...,0.032000,0.033,-0.005163,-0.025226,0.0,0.019205,-0.019136,0.007360,0.008068,0.0
1183742,-0.000142,-0.005846,-0.014684,0.004028,-0.005042,0.152,-0.095,0.002243,0.000259,-0.018253,...,-0.168000,-0.043,-0.005163,-0.025226,0.0,0.019205,-0.019136,-0.108000,0.008068,0.0
1183743,-0.000142,-0.005846,-0.014684,0.004028,-0.005195,-0.139,-0.184,0.002243,0.000259,-0.018253,...,0.141000,-0.004,-0.005163,-0.025226,0.0,0.019205,-0.019136,0.007303,0.008068,0.0
1183744,-0.000142,-0.005846,-0.014684,0.004028,-0.245000,-0.114,0.019,0.002243,0.000259,-0.018253,...,-0.154000,-0.001,-0.005163,-0.025226,0.0,0.019205,-0.018788,0.007303,0.008068,0.0


In [8]:
df_new = df_new[np.isfinite(df_new).all(1)]
df_new

,L1_S24_F1622,L1_S24_F1723,L1_S24_F1166,L1_S24_F1846,L0_S3_F96,L3_S33_F3859,L3_S33_F3857,L1_S24_F1844,L1_S24_F1609,L1_S24_F1474,...,L3_S30_F3574,L3_S30_F3769,L1_S25_F2925,L1_S25_F2484,L3_S34_F3880,L3_S41_F4020,L0_S14_F370,L2_S26_F3113,L1_S24_F844,Response
0,0.000018,0.000292,0.001569,-0.006682,-0.000358,-0.048,-0.102,0.001034,0.005514,0.001974,...,0.001711,-0.007,0.001233,0.016188,0.0,0.008414,-0.005228,-0.001004,-0.019477,0.0
1,0.000018,0.000292,0.001569,-0.006682,-0.000515,-0.210,-0.026,0.001034,0.005514,0.001974,...,0.033000,-0.024,0.001233,0.016188,0.0,0.008414,-0.324000,-0.001004,-0.019477,0.0
2,0.000018,0.000292,0.001569,-0.006682,-0.000515,0.236,0.191,0.001034,0.005514,0.001974,...,-0.147000,-0.026,0.001233,0.016188,0.0,0.008414,-0.324000,-0.001011,-0.019477,0.0
3,0.000018,0.000292,0.001569,-0.006682,-0.000515,0.165,0.031,0.001034,0.005514,0.001974,...,-0.054000,0.022,0.001233,0.016188,0.0,0.008414,-0.324000,-0.001011,-0.019477,0.0
4,0.000018,0.000292,0.001569,-0.006682,0.311000,0.147,-0.049,0.001034,0.005514,0.001974,...,0.119000,0.022,0.001233,0.016188,0.0,0.008746,-0.324000,-0.001011,-0.019477,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1183741,-0.000142,-0.038000,-0.014684,0.058000,-0.005042,0.054,0.033,0.261000,0.000259,-0.018253,...,0.032000,0.033,-0.005163,-0.025226,0.0,0.019205,-0.019136,0.007360,0.008068,0.0
1183742,-0.000142,-0.005846,-0.014684,0.004028,-0.005042,0.152,-0.095,0.002243,0.000259,-0.018253,...,-0.168000,-0.043,-0.005163,-0.025226,0.0,0.019205,-0.019136,-0.108000,0.008068,0.0
1183743,-0.000142,-0.005846,-0.014684,0.004028,-0.005195,-0.139,-0.184,0.002243,0.000259,-0.018253,...,0.141000,-0.004,-0.005163,-0.025226,0.0,0.019205,-0.019136,0.007303,0.008068,0.0
1183744,-0.000142,-0.005846,-0.014684,0.004028,-0.245000,-0.114,0.019,0.002243,0.000259,-0.018253,...,-0.154000,-0.001,-0.005163,-0.025226,0.0,0.019205,-0.018788,0.007303,0.008068,0.0


# Prepare data for CNN models

In [10]:
input_X = df_new.loc[:, df_new.columns != 'Response'].values  # converts the df to a numpy array
input_y = df_new['Response'].values

n_features = input_X.shape[1]  # number of features

In [11]:
n_features

58

In [12]:
input_y

array([0., 0., 0., ..., 0., 0., 0.])

In [13]:
input_X

array([[ 1.77304965e-05,  2.91970803e-04,  1.56896552e-03, ...,
        -5.22794118e-03, -1.00437158e-03, -1.94772727e-02],
       [ 1.77304965e-05,  2.91970803e-04,  1.56896552e-03, ...,
        -3.24000000e-01, -1.00437158e-03, -1.94772727e-02],
       [ 1.77304965e-05,  2.91970803e-04,  1.56896552e-03, ...,
        -3.24000000e-01, -1.01091703e-03, -1.94772727e-02],
       ...,
       [-1.41843972e-04, -5.84639498e-03, -1.46842105e-02, ...,
        -1.91362683e-02,  7.30278884e-03,  8.06818182e-03],
       [-1.41843972e-04, -5.84639498e-03, -1.46842105e-02, ...,
        -1.87876720e-02,  7.30278884e-03,  8.06818182e-03],
       [-1.41843972e-04, -5.84639498e-03, -1.46842105e-02, ...,
        -1.87876720e-02,  7.30278884e-03,  8.06818182e-03]])

In [14]:
n_features

58

### Divide the data into train, valid, and test

In [15]:
X_train, X_test, y_train, y_test = train_test_split(np.array(input_X), np.array(input_y), test_size=DATA_SPLIT_PCT, random_state=SEED)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=DATA_SPLIT_PCT, random_state=SEED)


In [16]:
X_train.shape

(757596, 58)

In [17]:
X_train

array([[ 4.60750853e-04,  1.99640288e-03, -1.78880000e-02, ...,
         6.66992187e-03, -1.34000000e-01,  2.94375000e-02],
       [-1.61966236e-04, -3.24414716e-04, -2.43518519e-02, ...,
         8.49693252e-03, -6.78588524e-03, -5.97435897e-02],
       [-1.01990527e-04,  3.06750498e-03, -7.64705882e-03, ...,
         8.01980198e-03, -2.72010309e-03,  1.83454545e-02],
       ...,
       [ 1.97183099e-04,  5.34050179e-04,  1.17777778e-02, ...,
        -9.53725490e-03, -3.04197107e-03, -9.31428571e-02],
       [ 3.08550186e-04, -1.06870229e-04,  9.57600000e-03, ...,
        -7.52131783e-03, -5.80897704e-03,  5.05918367e-02],
       [ 4.56928839e-04, -8.70550162e-04, -3.40150376e-02, ...,
         1.85525292e-02,  8.83883264e-03,  3.90327869e-02]])

In [18]:
np.array(input_y)

array([0., 0., 0., ..., 0., 0., 0.])

In [19]:
X_train_y0 = X_train[y_train==0]
X_train_y1 = X_train[y_train==1]

X_valid_y0 = X_valid[y_valid==0]
X_valid_y1 = X_valid[y_valid==1]

In [20]:
X_train_y0.shape

(753186, 58)

In [21]:
X_valid_y0.shape

(188271, 58)

In [22]:
len(X_train)

757596

In [23]:
num_classes=2

In [24]:
# Verify the reshaped data shape
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)


X_train shape: (757596, 58)
X_test shape: (236750, 58)


In [25]:
# Reshape the data to match the expected input shape of the model
X_train_reshaped = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test_reshaped = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))


In [26]:
# Verify the reshaped data shape
print("X_train shape:", X_train_reshaped.shape)
print("X_test shape:", X_test_reshaped.shape)


X_train shape: (757596, 58, 1)
X_test shape: (236750, 58, 1)


In [36]:
# Define the model
model = Sequential()
model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))



In [37]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [38]:
# Train the model
model.fit(X_train_reshaped, y_train, epochs=100, batch_size=32, validation_data=(X_test_reshaped, y_test))

Epoch 1/100
23675/23675 [==============================] - 121s 5ms/step - loss: 0.0369 - accuracy: 0.9942 - val_loss: 0.0352 - val_accuracy: 0.9943
Epoch 2/100
23675/23675 [==============================] - 119s 5ms/step - loss: 0.0362 - accuracy: 0.9942 - val_loss: 0.0355 - val_accuracy: 0.9943
Epoch 3/100
23675/23675 [==============================] - 117s 5ms/step - loss: 0.0360 - accuracy: 0.9942 - val_loss: 0.0350 - val_accuracy: 0.9943
Epoch 4/100
23675/23675 [==============================] - 119s 5ms/step - loss: 0.0359 - accuracy: 0.9942 - val_loss: 0.0350 - val_accuracy: 0.9943
Epoch 5/100
23675/23675 [==============================] - 129s 5ms/step - loss: 0.0360 - accuracy: 0.9942 - val_loss: 0.0350 - val_accuracy: 0.9943
Epoch 6/100
23675/23675 [==============================] - 129s 5ms/step - loss: 0.0360 - accuracy: 0.9942 - val_loss: 0.0349 - val_accuracy: 0.9943
Epoch 7/100
23675/23675 [==============================] - 129s 5ms/step - loss: 0.0360 - accuracy: 0.9942

In [39]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test_reshaped, y_test, verbose=0)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)


Test Loss: 0.03499554470181465
Test Accuracy: 0.9943400025367737


In [40]:
# Make predictions
predictions = model.predict(X_test_reshaped)

7399/7399 [==============================] - 14s 2ms/step


In [41]:
predictions

array([[0.00484208],
       [0.00484208],
       [0.00484208],
       ...,
       [0.00484208],
       [0.00484208],
       [0.00484208]], dtype=float32)

In [42]:
predicted_labels = np.round(predictions)  # Round the predictions to obtain binary labels

In [43]:
# Print confusion matrix and classification report
cm = confusion_matrix(y_test, predicted_labels)
print('Confusion Matrix:')
print(cm)


Confusion Matrix:
[[235410      0]
 [  1340      0]]


In [44]:
print('Classification Report:')
print(classification_report(y_test, predicted_labels))

Classification Report:


/Users/chathurangishyalika/opt/anaconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         0.0       0.99      1.00      1.00    235410
         1.0       0.00      0.00      0.00      1340

    accuracy                           0.99    236750
   macro avg       0.50      0.50      0.50    236750
weighted avg       0.99      0.99      0.99    236750



/Users/chathurangishyalika/opt/anaconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/chathurangishyalika/opt/anaconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
